In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("../data/인터뷰/ckmk_d_bm_f_n_169087.json")

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# read_json을 통해 json파일을 로드했는데? 데이터 형태가 데이터프레임에 최적화x
# 답변의 데이터, 답변의 요약데이터를 추출 -> kobart 모델을 이용해서 요약 학습
# json 파일을 dict 형태로 변환 -> 데이터 추출
import json

In [ ]:
# 파일을 로드해서 dict 형태로 변환 
with open("../data/인터뷰/ckmk_d_bm_f_n_169087.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [11]:
f2 = open("../data/인터뷰/ckmk_d_bm_f_n_169087.json", 'r', encoding='utf-8')
data2 = json.load(f2)

In [ ]:
type(data)

In [ ]:
from pprint import pprint

In [ ]:
print(data)

In [13]:
pprint(data)

{'dataSet': {'answer': {'emotion': [],
                        'intent': [{'category': 'attitude',
                                    'expression': '',
                                    'text': ''}],
                        'raw': {'text': '저는 저 혼자 시간을 보내는 것을 우선적으로 하고요. 예를 들어서 '
                                        '혼자 산책을 한다든지 아니면 혼자 카페를 간다든지 아니면 혼자 코인 '
                                        '노래방을 간다든지 해서 스트레스를 해소하는 편입니다. 물론 남들과 '
                                        '만나면서 시간을 보내는 것도 굉장히 좋아하지만요. 아무래도 내 '
                                        '혼자만의 시간이 있어야 어느 정도 생각 정리도 되고 그러면서 내 '
                                        '안에 있던 응어리도 절로 풀러지는 그런 것을 경험하게 되었습니다. '
                                        '그리고 코인 노래방이나 이런 곳에 가면은 아무래도 제가 노래 부르는 '
                                        '것도 좋아하고 그리고 노래를 부르면서 그런 소리가 웅웅거리고 또 '
                                        '소리를 지른다는 거에 대해서 해소감도 있는 것 같고요. 그래서 저는 '
                                        '일단 혼자만의 시간을 보내는데 뭐 혼자서 노래방을 가거나 아니면 '

## kobart 복습 
1. 인터뷰 폴더 안에 있는 json 파일들을 로드 
    - 답변에 대한 데이터들을 하나의 리스트로 생성 
    - 답변 요약에 대한 데이터들을 하나의 리스트로 생성 
2. 위에서 생성된 데이터들중 train데이터는 처음부터 100번째 데이터까지 
3. 마지막에서 10번째부터 마지막까지의 데이터를 validation 데이터로 사용
4. train, test 데이터를 DatasetDict 형태로 변환 
5. tokenizer와 model을 로드하여 데이터를 학습하고 검증 
    - input의 최대 사이즈 : 512
    - outpit의 최대 사이즈 : 256
6. test 데이터를 생성 : 원본의 리스트에서 200번째의 데이터를 이용하여 요약을 생성

In [14]:
# json 파일의 목록을 생성 
import os 
from glob import glob

In [19]:
file_path = "../data/인터뷰/"
file_list = os.listdir(file_path)

In [20]:
file_list2 = glob("../data/인터뷰/*.json")

In [21]:
for file in file_list:
    # os 라이브러리를 이용하여 파일의 목록을 불러왔을때
    path = file_path + file
    print(path)
    break

../data/인터뷰/ckmk_d_bm_f_n_169087.json


In [24]:
# answer의 원문과 요약 텍스트들을 저장할수 있는 빈 리스트 생성
answers, summarys = [], []
for file in file_list2:
    # print(file)
    # break
    try:
        with open(file, 'r', encoding='utf-8') as f:
            dict_data = json.load(f)
            answer = dict_data['dataSet']['answer']['raw']['text']
            summary = dict_data['dataSet']['answer']['summary']['text']
            answers.append(answer)
            summarys.append(summary)
    except Exception as e:
        print(e)
        continue
    


Invalid control character at: line 23 column 135 (char 755)
Invalid control character at: line 23 column 571 (char 1132)
Invalid control character at: line 23 column 433 (char 1029)


In [25]:
len(answers)

12801

In [26]:
# 학습에 걸리는 시간이 길기때문에 일부분만 확인
train_docs = answers[ : 100]
train_sums = summarys[ : 100]
valid_docs = answers[-10 : ]
valid_sums = summarys[-10 : ]

In [27]:
print( len(train_docs), len(train_sums) )
print( len(valid_docs), len(valid_sums) )

100 100
10 10


In [28]:
import numpy as np 
from datasets import Dataset, DatasetDict
import evaluate

from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments
)

In [29]:
model_name = "gogamja/kobart-base-v2"

In [30]:
# DatasetDict 생성 
raw_ds = DatasetDict(
    {
        'train' : Dataset.from_dict(
            {
                'document' : train_docs, 
                'summary' : train_sums
            }
        ), 
        'validation' : Dataset.from_dict(
            {
                'document' : valid_docs, 
                'summary' : valid_sums
            }
        )
    }
)